In [ ]:
import pickle
import cobrame
from cobrame.io.json import load_reduced_json_me_model, load_json_me_model
from multiprocessing import Pool
import pandas as pd
import numpy as np
import cobra.test
from collections import defaultdict
from sympy import Basic, Symbol
from datetime import datetime

In [ ]:
# Import the iJL1678b-ME model of E.coli K-12 MG1655
with open('./me_models/iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

In [ ]:
# Solve the ME-model using either SoPlex or qMINOS
def solve_me_model(me, max_mu, precision=1e-6, min_mu=0, using_soplex=True,
                  compiled_expressions=None):
    if using_soplex:
        from cobrame.solve.algorithms import binary_search
        binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                      compiled_expressions=compiled_expressions)
    else:
        from qminospy.me1 import ME_NLP1
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use bisection for now (until the NLP formulation is worked out)
        muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu)
        if me.solution:
            me.solution.f = me.solution.x_dict['biomass_dilution']

In [ ]:
# Create a iJO1366 M-model of E.coli to use it's BOF
ijo = cobra.test.create_test_model('ecoli')

# Extract the growth normalized biomass composition of the cell from the solution and return it as a list.
def find_biomass (model, solution):
    
    if solution:
        model.solution = solution
    me_demand = defaultdict(float)
    x_dict = model.solution.x_dict

    # These are reactions that incorporate metabolites into biomass
    skip_list = ['SummaryVariable', 'ComplexFormation',
                 'TranscriptionReaction', 'TranslationReaction']

    growth_rate = model.solution.x_dict['biomass_dilution'] # "biomass_dilution" reaction is the pseudo-reaction representing growth rate
    mu = Symbol('mu')

    biomass_rxn = ijo.reactions.Ec_biomass_iJO1366_WT_53p95M # The BOF used for finding the metabolites of interest
    for met_id in biomass_rxn.metabolites:
        if met_id.id == "lipopb_c":
            met = model.metabolites.get_by_id("lipoate_c") # lipoate has different IDs in the iJO1366 and the iJL1678b models
        else:
            met = model.metabolites.get_by_id(met_id.id)
        for r in met.reactions:
            if r.__class__.__name__ not in skip_list: # We filter out all the reactions that are not synthesis reactions
                stoich = r._metabolites[met]
                if isinstance(stoich, Basic):                  
                    mu_symbol = None
                    for symbol in stoich.free_symbols:
                        if symbol.name == 'mu': 
                            mu_symbol = symbol
                            break
                    
                    if mu_symbol:
                        stoich = stoich.subs(mu_symbol, growth_rate) # The "mu" symbol gets replaced with growth rate
                        
                me_demand[met_id.id] += (
                        x_dict[r.id] * stoich / growth_rate) # We find the fluxes of the synthesis reactions and growth normalize them
    return me_demand

In [ ]:
# Solves the model under certain glucose and ammonium availabilities and returns the growth rate, nutrient constraints 
# and biomass constituents
def solve_and_save(model, glc, nh4):
    model.reactions.EX_glc__D_e.lower_bound = glc
    model.reactions.EX_nh4_e.lower_bound = nh4
    solve_me_model(model, 1., min_mu = .1, precision=1e-4, using_soplex=False)
    if model.solution:
        biomass = find_biomass(model, solution= model.solution)
        return (model.solution.x_dict['biomass_dilution'], glc, nh4, biomass)

In [ ]:
#Merges the biomass defaultdict into the list
def sort_result(result):
    sorted_list = []
    sorted_list.extend([result[0], result[1], result[2]])
    for bio_component in result[3].values():
        sorted_list.append(bio_component)
    return sorted_list

In [ ]:
# Function for creating the glucose and ammonium constraint data to be processed
def create_tasks(model, glc_min=-10, glc_max=-0, nh4_min=-10, nh4_max=-0, step_size=0.2):
    glc_values = list(np.arange(glc_min, glc_max, step_size))
    nh4_values = list(np.arange(nh4_min, nh4_max, step_size))
    
    tasks = [(model, glc, nh4) for glc in glc_values for nh4 in nh4_values]
    return tasks

In [ ]:
# Create the constraints specific for our study and batch them up for running 6 simulations at a time
tasks = create_tasks(me, glc_min=-10, glc_max=-0, nh4_min=-10, nh4_max=-0, step_size=0.2)
new_lists = [tasks[i:i + 6] for i in range(0, len(tasks), 6)]
len(new_lists)

In [ ]:
# Split the data into batches for simulations
batch1 = new_lists[0:25]
batch2 = new_lists[25:50]
batch3 = new_lists[50:75]
batch4 = new_lists[75:100]
batch5 = new_lists[100:125]
batch6 = new_lists[125:150]
batch7 = new_lists[150:175]
batch8 = new_lists[175:200]
batch9 = new_lists[200:225]
batch10 = new_lists[225:250]
batch11 = new_lists[250:275]
batch12 = new_lists[275:300]
batch13 = new_lists[300:350]
batch14 = new_lists[350:375]
batch15 = new_lists[375:400]
batch16 = new_lists[400:416]

In [ ]:
# Uses multiprocessing to parallelize simulations, converts the results into a DataFrame and writes them to a CSV file
for i, commit in enumerate(batch1):
    if __name__ == "__main__":
        with Pool() as p:
            results = p.starmap(solve_and_save, commit)

        sorted_results = []
        for result in results:
            sorted_results.append(sort_result(result))


        df = pd.DataFrame(sorted_results, columns=['growth_rate', 'glc_bound', 'nh4_bound', 'pe161_c', 'murein4p4p_p', 
    'pg161_c', 'pe161_p', 'murein4px4p_p', 'chor_c', 'pg161_p', 'trp__L_c', 'glu__L_c', '4fe4s_c', 'ni2_c', 'malcoa_c', 
    'udcpdp_c', 'gtp_c', 'nadp_c', 'h2o_c', 'thmpp_c', '5mthf_c', 'bmocogdp_c', 'adocbl_c', '2dmmql8_c', 'succoa_c', 
    'nh4_c', 'leu__L_c', 'q8h2_c', 'enter_c', 'cobalt2_c', 'cu2_c', 'pydx5p_c', 'ca2_c', 'asn__L_c', 'pe160_p', 'pg160_p', 
    'asp__L_c', 'dctp_c', 'pe160_c', 'pg160_c', 'coa_c', 'fe2_c', 'mg2_c', 'glycogen_c', 'spmd_c', 'ala__L_c', 
    'sheme_c', 'ptrc_c', 'arg__L_c', 'thf_c', 'tyr__L_c', 'thr__L_c', 'ctp_c', 'ser__L_c', 'dttp_c', 'fad_c', 'atp_c', 
    'gln__L_c', 'pheme_c', 'btn_c', 'gthrd_c', 'fe3_c', 'met__L_c', 'lys__L_c', 'clpn181_p', 'amet_c', 'ribflv_c', 'mobd_c', 
    '2fe2s_c', 'pg181_c', 'accoa_c', 'pg181_p', 'mocogdp_c', 'murein3px4p_p', 'cys__L_c', '10fthf_c', 'murein3p3p_p', 
    'dgtp_c', 'clpn161_p', 'mlthf_c', 'colipa_e', 'murein4px4px4p_p', 'pe181_c', 'his__L_c', 'val__L_c', 'utp_c', 'pe181_p', 
    'k_c', 'hemeO_c', 'ile__L_c', 'so4_c', 'zn2_c', 'cl_c', 'nadph_c', 'nad_c', 'mn2_c', 'pro__L_c', 'lipopb_c', 'nadh_c', 
    'phe__L_c', 'clpn160_p', 'mococdp_c', 'gly_c', 'mql8_c', 'datp_c', 'ppi_c', 'pi_c', 'h_c', 'adp_c'])


        with open('results.csv', 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False)
            
        print(f"Progress: {i+1}/{len(batch1)} commits processed. Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")